Dataset link

https://chanzuckerberg.github.io/cellxgene-census/notebooks/api_demo/census_datasets.html

In [1]:
!python --version

Python 3.10.18


In [2]:
import cellxgene_census

census = cellxgene_census.open_soma(census_version="2025-01-30")
census_datasets = census["census_info"]["datasets"].read().concat().to_pandas()

# for convenience, indexing on the soma_joinid which links this to other census data.
census_datasets = census_datasets.set_index("soma_joinid")

census_datasets

,citation,collection_id,collection_name,collection_doi,collection_doi_label,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,,,,
0,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,4eb29386-de81-452f-b3c0-e00844e8c7fd,f76861bb-becb-4eb7-82fc-782dc96ccc7f,Spatial transcriptomics in mouse: Puck_191112_05,4eb29386-de81-452f-b3c0-e00844e8c7fd.h5ad,10888
1,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,78d59e4a-82eb-4a61-a1dc-da974d7ea54b,7d7ec1b6-6e3f-4aaa-9442-4b22f3424396,Spatial transcriptomics in mouse: Puck_191112_08,78d59e4a-82eb-4a61-a1dc-da974d7ea54b.h5ad,10250
2,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,add5eb84-5fc9-4f01-982e-a346dd42ee82,de54aed8-4f73-48f6-9229-418a840e2d82,Spatial transcriptomics in mouse: Puck_191109_20,add5eb84-5fc9-4f01-982e-a346dd42ee82.h5ad,12906
3,Publication: https://doi.org/10.1016/j.isci.20...,8e880741-bf9a-4c8e-9227-934204631d2a,High Resolution Slide-seqV2 Spatial Transcript...,10.1016/j.isci.2022.104097,Marshall et al. (2022) iScience,b020294c-ab82-4547-b5a7-63d8ffa575ed,abe4fce1-0859-4a56-ad1e-734d79f0e6c8,Spatial transcriptomics in mouse: Puck_191112_13,b020294c-ab82-4547-b5a7-63d8ffa575ed.h5ad,15161
4,Publication: https://doi.org/10.1038/s41591-02...,a96133de-e951-4e2d-ace6-59db8b3bfb1d,HTAN/HTAPP Broad - Spatio-molecular dissection...,10.1038/s41591-024-03215-z,Klughammer et al. (2024) Nat Med,d7476ae2-e320-4703-8304-da5c42627e71,863fc5e4-bd4a-4681-9c3d-0ee7ef54e327,HTAPP-330-SMP-1082 scRNA-seq,d7476ae2-e320-4703-8304-da5c42627e71.h5ad,565
...,...,...,...,...,...,...,...,...,...,...
1568,Publication: https://doi.org/10.1038/s41586-02...,45d5d2c3-bc28-4814-aed6-0bb6f0e11c82,A single-cell transcriptional timelapse of mou...,10.1038/s41586-024-07069-w,Qiu et al. (2024) Nature,dcfa2614-7ca7-4d82-814c-350626eccb26,3002a659-a1a9-4406-9976-99e658e1fbb5,Major cell cluster: Mesoderm,dcfa2614-7ca7-4d82-814c-350626eccb26.h5ad,3267338
1569,Publication: https://doi.org/10.1126/science.a...,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,The Tabula Sapiens Consortium* et al. (2022) S...,53d208b0-2cfd-4366-9866-c3c6114081bc,10df7690-6d10-4029-a47e-0f071bb2df83,Tabula Sapiens - All Cells,53d208b0-2cfd-4366-9866-c3c6114081bc.h5ad,1136218
1570,Publication: https://doi.org/10.1038/s41586-02...,45d5d2c3-bc28-4814-aed6-0bb6f0e11c82,A single-cell transcriptional timelapse of mou...,10.1038/s41586-024-07069-w,Qiu et al. (2024) Nature,dcfd4feb-18a3-4b30-81d7-1b0c544a8ab3,3817734b-0f82-433b-8c38-55b214200fff,Whole dataset: Raw counts only,dcfd4feb-18a3-4b30-81d7-1b0c544a8ab3.h5ad,11441407


In [20]:
def format_description_prompt(row):
    prompt = f"""
Please summarize the following RNA-seq dataset clearly and concisely in 100–150 words, and also identify the experimental technology used.

Focus your summary on:
- The tissue or organ studied
- Organism
- Number of cells and samples (if available)
- Sample conditions such as health status, age, sex, or strain
- Study purpose and biological context (e.g. disease studied, pathways analyzed)
- Major annotated or observed cell types

Then identify the **specific sequencing technology** used in the experiment (e.g. Smart-seq2, 10x Genomics Chromium, Slide-seqV2, Drop-seq, Microwell-seq, sci-RNA-seq, etc).
Do not guess based on publication year or general assumptions. Instead, carefully read the provided title, citation, and collection description for evidence.
Do **not** use uncertain language like "likely", "possibly", or "based on". Only output a specific technology name if the content supports it through wording, phrasing, or indirect description.
If the technique is not clearly indicated, leave the field blank.

You may use clues from the citation, title, or collection name to infer the likely sequencing technology.
For example, a 2015 study using tumor biopsy samples likely used Smart-seq2 or Drop-seq. Studies by 10x Genomics likely used 10x Chromium.

Strictly format your response as:
Description: <one-paragraph English summary>
Technology: <exact technology name only — no extra words, no sentences, no parentheses>

Do not include any additional text, explanations, or bullet points.

Dataset Title: {row['dataset_title']}
Citation: {row['citation']}
DOI: {row['collection_doi']} ({row['collection_doi_label']})
Collection Name: {row['collection_name']} (ID: {row['collection_id']})
Cell Count: {row['dataset_total_cell_count']:,} cells

Describe this dataset in natural language:
""".strip()
    return prompt

In [23]:
import requests, os, json

def openrouter_chat(model: str, prompt: str):
    resp = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
            "Content-Type": "application/json"
        },
        data=json.dumps({
            "model": model,
            "messages": [{"role": "user", "content": prompt}]
        })
    )
    return resp.json()

# Example usage of the OpenRouter API to summarize a dataset description
model = "deepseek/deepseek-r1-0528:free"
query = format_description_prompt(census_datasets.iloc[0])
print("User: ")
print(query)

result = openrouter_chat(model, query)
try:
    message = result["choices"][0]["message"]["content"]
    print(f"\nAssistant ({model}): ")
    print(message)
except KeyError:
    print("Error in response format:", result)
    # raise ValueError("Unexpected response format from OpenRouter API")

User: 
Please summarize the following RNA-seq dataset clearly and concisely in 100–150 words, and also identify the experimental technology used.

Focus your summary on:
- The tissue or organ studied
- Organism
- Number of cells and samples (if available)
- Sample conditions such as health status, age, sex, or strain
- Study purpose and biological context (e.g. disease studied, pathways analyzed)
- Major annotated or observed cell types

Then identify the **specific sequencing technology** used in the experiment (e.g. Smart-seq2, 10x Genomics Chromium, Slide-seqV2, Drop-seq, Microwell-seq, sci-RNA-seq, etc).
Do not guess based on publication year or general assumptions. Instead, carefully read the provided title, citation, and collection description for evidence.
Do **not** use uncertain language like "likely", "possibly", or "based on". Only output a specific technology name if the content supports it through wording, phrasing, or indirect description.
If the technique is not clearly 

In [24]:
import pandas as pd

# The census_datasets DataFrame is large, this is a subset for demonstration
df = census_datasets[0:10]

def generate_description(row, model="deepseek/deepseek-r1-0528-qwen3-8b:free"):
    prompt = format_description_prompt(row)
    result = openrouter_chat(model, prompt)
    try:
        return result["choices"][0]["message"]["content"].strip()
    except (KeyError, IndexError) as e:
        return f"[Error retrieving description: {e}]"

df['raw_response'] = df.apply(generate_description, axis=1)

def parse_response(text):
    if text.startswith("[Error"):
        return pd.Series({'description': text, 'technology': "Error"})
    
    parts = text.split("Technology:")
    if len(parts) == 2:
        description = parts[0].replace("Description:", "").strip()
        technology = parts[1].strip()
    else:
        description = text.strip()
        technology = "Unknown"
    return pd.Series({'description': description, 'technology': technology})

parsed = df['raw_response'].apply(parse_response)

new_df = pd.DataFrame({
    'dataset_title': df['dataset_title'],
    'collection_doi': df['collection_doi'],
    'dataset_total_cell_count': df['dataset_total_cell_count'],
    'description': parsed['description'],
    'technology': parsed['technology']
})

new_df.head(10)

/tmp/ipykernel_1007/2837676445.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['raw_response'] = df.apply(generate_description, axis=1)


,dataset_title,collection_doi,dataset_total_cell_count,description,technology
soma_joinid,,,,,
0,Spatial transcriptomics in mouse: Puck_191112_05,10.1016/j.isci.2022.104097,10888,This dataset characterizes the transcriptome o...,Slide-seqV2
1,Spatial transcriptomics in mouse: Puck_191112_08,10.1016/j.isci.2022.104097,10250,This dataset provides spatial transcriptomics ...,Slide-seqV2
2,Spatial transcriptomics in mouse: Puck_191109_20,10.1016/j.isci.2022.104097,12906,This dataset captures gene expression patterns...,Slide-seqV2
3,Spatial transcriptomics in mouse: Puck_191112_13,10.1016/j.isci.2022.104097,15161,This dataset represents a high-resolution spat...,Slide-seqV2
4,HTAPP-330-SMP-1082 scRNA-seq,10.1038/s41591-024-03215-z,565,This dataset includes single-cell RNA sequenci...,
5,Spatial transcriptomics in mouse: Puck_191109_14,10.1016/j.isci.2022.104097,12351,This dataset offers high-resolution spatial tr...,Slide-seqV2
6,Healthy human liver: B cells,10.1002/hep4.1854,146,This dataset comprises single-nucleus RNA sequ...,
7,thymus scRNA-seq atlas - myeloid p2 subset,10.1038/s41586-024-07944-6,843,"This dataset explores the human thymus, a cent...",
8,Spatial transcriptomics in mouse: Puck_191109_18,10.1016/j.isci.2022.104097,19156,This dataset explores spatial gene expression ...,Slide-seqV2


In [25]:
df['raw_response'][1]

'Description:\nThis dataset provides spatial transcriptomics data from mouse tissue, comparing normal versus diseased states, focusing on disease-specific cellular neighborhoods, cell states, and pathways involved in pathology. It analyzed approximately 10,250 cells obtained from distinct tissue regions associated with different disease conditions, enabling a detailed examination of the spatio-temporal dynamics of gene expression changes linked to pathological processes. The study aims to identify localized transcriptional programs and interactions relevant to disease mechanisms within the tissue architecture.\n\nTechnology: Slide-seqV2'